In [3]:
from ete3 import NCBITaxa
ncbi = NCBITaxa()

NCBI database not present yet (first time used?)
Done. Parsing...


Loading node names...
1548246 names loaded.
183362 synonyms loaded.
Loading nodes...
1548246 nodes loaded.
Linking nodes...
Tree is loaded.
Updating database: /home/gamran/.etetoolkit/taxa.sqlite ...
 1548000 generating entries... 

Inserting synonyms:      20000 


Uploading to /home/gamran/.etetoolkit/taxa.sqlite



Inserting taxid merges:  40000 

Inserting taxids:       10000 

Inserting taxids:       1545000 

In [4]:
#Constructing dataframes for each individual blasted barcode
import os
os.chdir('/home/gamran/blast/')
outdir = '/home/gamran/analysis/'
os.curdir

'.'

In [18]:
blast_input_dir = '/home/gamran/blast'
blast_files_in_folder = [x for x in list(os.walk(blast_input_dir, topdown=False))][1][2]
NCBI_files_in_folder = sorted([x for x in blast_files_in_folder if 'NCBI' in x])

['1212_1D_barcoding_Wagga_BC00.WW_19121016.noblasthit.fa.NCBI04052016.001.besthit',
 '1212_1D_barcoding_Wagga_BC00.WW_19121016.noblasthit.fa.NCBI04052016.besthit',
 '1212_1D_barcoding_Wagga_BC01.WW_19121016.noblasthit.fa.NCBI04052016.001.besthit',
 '1212_1D_barcoding_Wagga_BC01.WW_19121016.noblasthit.fa.NCBI04052016.besthit',
 '1212_1D_barcoding_Wagga_BC02.WW_19121016.noblasthit.fa.NCBI04052016.001.besthit',
 '1212_1D_barcoding_Wagga_BC02.WW_19121016.noblasthit.fa.NCBI04052016.besthit',
 '1212_1D_barcoding_Wagga_BC03.WW_19121016.noblasthit.fa.NCBI04052016.001.besthit',
 '1212_1D_barcoding_Wagga_BC03.WW_19121016.noblasthit.fa.NCBI04052016.besthit',
 '1212_1D_barcoding_Wagga_BC04.WW_19121016.noblasthit.fa.NCBI04052016.besthit',
 '1212_1D_barcoding_Wagga_BC05.WW_19121016.noblasthit.fa.NCBI04052016.001.besthit',
 '1212_1D_barcoding_Wagga_BC05.WW_19121016.noblasthit.fa.NCBI04052016.besthit',
 '1212_1D_barcoding_Wagga_BC06.WW_19121016.noblasthit.fa.NCBI04052016.001.besthit',
 '1212_1D_barcod

In [30]:
#Creating a dictionary for NCBI blasted barcodes - key: 'barcode', value: related NCBI blast dataframe
import pandas as pd

NCBI_dfs_dict = {}
for file_name in NCBI_files_in_folder:
    if '.001.' in file_name:
        continue
    elif '.besthit' in file_name:
        key = '' #Safety 
        key = file_name.split('.')[0][-4:] # Try to make a pattern match, not an index hard code
        NCBI_dfs_dict[key] = pd.read_table(file_name, delim_whitespace=False, 
                                           names=['qseqid', 'sseqid', 'evalue', 'bitscore', 'length', 
                                                  'pident', 'nident', 'sgi', 'sacc', 'staxids', 'sscinames', 
                                                  'scomnames', 'sskingdoms', 'barcode'])
    continue

In [32]:
#Assigning barcodes to the barcode column based on dict key
for bcs, dfs in NCBI_dfs_dict.items():
    for i in range(0, len(dfs)):
        dfs.loc[i, 'barcode'] = bcs

In [37]:
#Joining the dataframes for individual barcodes together
NCBI_df_list = []
for bcs in NCBI_dfs_dict.keys():
    NCBI_df_list.append(NCBI_dfs_dict[bcs])
    
joint_NCBI_df = pd.concat(NCBI_df_list, axis=0, ignore_index=True)
joint_NCBI_df.sort_values('barcode', ascending=True, inplace=True)
joint_NCBI_df.reset_index(drop=True, inplace=True)

In [66]:
joint_NCBI_df
#joint_NCBI_df[joint_NCBI_df.barcode == 'BC02']

,qseqid,sseqid,evalue,bitscore,length,pident,nident,sgi,sacc,staxids,sscinames,scomnames,sskingdoms,barcode
0,b694e6b0-be3f-41d4-a1ac-960beaa1e3cd_Basecall_...,gi|1008889473|gb|CP014768.1|,0.000000e+00,1618.0,1512,87.37,1321,1008889473,CP014768,1813821,Shigella sp. PAMC 28760,Shigella sp. PAMC 28760,Bacteria,BC00
1,3e9eaf56-497e-4892-b9e2-b28700044e69_Basecall_...,gi|1051181|gb|U37692.1|CVU37692,0.000000e+00,1792.0,2255,83.15,1875,1051181,U37692,43300,Cloning vector lambda TXF97,Cloning vector lambda TXF97,NaN,BC00
2,3fc3a37e-0151-4ec6-95c9-312e779bd514_Basecall_...,gi|1051181|gb|U37692.1|CVU37692,0.000000e+00,1195.0,1155,86.84,1003,1051181,U37692,43300,Cloning vector lambda TXF97,Cloning vector lambda TXF97,NaN,BC00
3,400652b7-6dff-49de-bf65-1847b08fbffb_Basecall_...,gi|215104|gb|J02459.1|LAMCG,0.000000e+00,3369.0,2587,91.15,2358,215104,J02459,10710,Enterobacteria phage lambda,Enterobacteria phage lambda,Viruses,BC00
4,41c35e20-2205-405b-adab-75132600e591_Basecall_...,gi|1051181|gb|U37692.1|CVU37692,0.000000e+00,1297.0,1373,85.14,1169,1051181,U37692,43300,Cloning vector lambda TXF97,Cloning vector lambda TXF97,NaN,BC00
5,4248f4fe-34e0-4ce5-ade8-d05002a3058d_Basecall_...,gi|445198867|gb|CP004045.1|,0.000000e+00,1327.0,1402,85.24,1195,445198867,CP004045,1282356,Pseudomonas poae RE*1-1-14,Pseudomonas poae RE*1-1-14,Bacteria,BC00
6,4616a9eb-c856-4803-9e74-de9c48ca4486_Basecall_...,gi|1008889473|gb|CP014768.1|,0.000000e+00,1044.0,896,89.06,798,1008889473,CP014768,1813821,Shigella sp. PAMC 28760,Shigella sp. PAMC 28760,Bacteria,BC00
7,4902fe2e-72ea-4eb4-ad5c-51dd2d7d1e67_Basecall_...,gi|1051181|gb|U37692.1|CVU37692,0.000000e+00,1537.0,1757,84.18,1479,1051181,U37692,43300,Cloning vector lambda TXF97,Cloning vector lambda TXF97,NaN,BC00
8,492645b7-63c1-42ca-a901-ebe6a6643870_Basecall_...,gi|1051181|gb|U37692.1|CVU37692,0.000000e+00,743.0,1529,77.50,1185,1051181,U37692,43300,Cloning vector lambda TXF97,Cloning vector lambda TXF97,NaN,BC00
9,49fb902b-070e-4e76-b158-f15952ed933e_Basecall_...,gi|1002167280|gb|CP012733.1|,4.000000e-56,230.0,210,87.62,184,1002167280,CP012733,374923,Shigella flexneri 1a,Shigella flexneri 1a,Bacteria,BC00


In [73]:
joint_NCBI_pivot_table = joint_NCBI_df.pivot_table(values='nident', index='scomnames', columns='barcode', aggfunc='count', fill_value=0, margins=True)
joint_NCBI_pivot_table.index.name = None
joint_NCBI_pivot_table.columns.name = None


Achromobacter xylosoxidans                             1.0
Acidosasa purpurea                                     0.0
Actinoplanes sp. SE50/110                              0.0
Acytostelium subglobosum LB1                           0.0
Aegilops crassa                                        0.0
Aegilops geniculata                                    1.0
Aegilops speltoides                                    1.0
Aegilops tauschii                                      2.0
African malaria mosquito                              13.0
Agrobacterium radiobacter K84                          0.0
Albugo laibachii Nc14                                  0.0
Alicycliphilus denitrificans BC                        0.0
Altererythrobacter epoxidivorans                       0.0
Alternaria brassicicola                                1.0
Anthoxanthum odoratum                                  0.0
Aristolochia macrophylla                               0.0
Arthrobacter arilaitensis                              1

In [94]:
def ten_best_species(bcs):
    print(joint_NCBI_pivot_table[bcs].sort_values(axis=0, ascending=False).head(n=10))
    
for bcs in sorted(list(NCBI_dfs_dict.keys())):
    ten_best_species(bcs)
    print()

All                                        134.0
Cloning vector lambda TXF97                 56.0
Shigella sp. PAMC 28760                     45.0
Pseudomonas poae RE*1-1-14                   8.0
Enterobacteria phage lambda                  7.0
Escherichia coli                             4.0
Pseudomonas syringae pv. syringae HS191      4.0
Zymoseptoria tritici                         2.0
Shigella flexneri 1a                         2.0
Bipolaris sorokiniana ND90Pr                 1.0
Name: BC00, dtype: float64

All                                                        451.0
Cloning vector lambda TXF97                                239.0
Shigella sp. PAMC 28760                                    122.0
Enterobacteria phage lambda                                 43.0
Escherichia coli                                            12.0
Shigella flexneri 1a                                         5.0
Salmonella enterica subsp. enterica serovar Typhimurium      5.0
Enterobacteria phage CUS-3 

['BC05', 'BC06', 'BC01', 'BC04', 'BC00', 'NB00', 'BC02', 'BC03']

In [48]:
#No of unique species identified across all barcodes
len(joint_NCBI_df.staxids.unique())

560

In [45]:
NCBI_dfs_dict['NB00'].tail()

,qseqid,sseqid,evalue,bitscore,length,pident,nident,sgi,sacc,staxids,sscinames,scomnames,sskingdoms,barcode
42,e4e15885-c061-4f32-b847-cca6d6858ce3_Basecall_...,gi|1051181|gb|U37692.1|CVU37692,0.000000e+00,1371.0,1111,89.92,999,1051181,U37692,43300,Cloning vector lambda TXF97,Cloning vector lambda TXF97,NaN,NB00
43,eb0b9dc3-d98f-4f96-a45b-e0e3afda927c_Basecall_...,gi|1008889473|gb|CP014768.1|,1.000000e-120,444.0,383,88.77,340,1008889473,CP014768,1813821,Shigella sp. PAMC 28760,Shigella sp. PAMC 28760,Bacteria,NB00
44,ef721feb-2406-4593-9e13-395e3c987f08_Basecall_...,gi|1001869111|gb|CP014522.1|,1.000000e-44,191.0,162,88.27,143,1001869111,CP014522,562,Escherichia coli,Escherichia coli,Bacteria,NB00
45,f7695fba-01fa-4ca7-b6fb-2faa56221862_Basecall_...,gi|1051181|gb|U37692.1|CVU37692,0.000000e+00,1007.0,918,87.91,807,1051181,U37692,43300,Cloning vector lambda TXF97,Cloning vector lambda TXF97,NaN,NB00
46,fb00379a-8abf-46b5-9b1b-5db8ab49e838_Basecall_...,gi|970580326|gb|CP013183.1|,0.000000e+00,1107.0,1087,86.11,936,970580326,CP013183,199201,Pseudomonas syringae pv. lapsa,Pseudomonas syringae pv. lapsa,Bacteria,NB00
